##### Тестирование параметра OVERDUE_FLG наличия просрочки по договору на определенную дату


сравнение  "data/external/contracts_overdue_tsal_for_test.pkl" - просроченные договоры ЦАЛ с "data/interm/test/overdue_contracts.pkl" - рассчитанные просроченные договоры

In [1]:
pre = '../../' # путь до главной директории проекта
%run ../../params.ipynb #параметры как логин пароль к БИ, параметры расчёта ( в нашем случае их нет)
%run ../init.ipynb #подгрузить нужные библиотеки

In [2]:
# сюда таблицы служебные будем кидать, для отчетов
name = pre + "data/interm/auxilary/test_contract_ovd_cnt.xlsx"
writer = ExcelWriter(name)

In [3]:
name = pre + "data/external/contracts_overdue_tsal_for_test.pkl"
contracts_overdue_tsal = pd.read_pickle(name, compression={'method':'zip'})

print(len(contracts_overdue_tsal))

num, duplicates = identify_duplicates(contracts_overdue_tsal, 'CONTRACT_KEY')
Check_dupl(num, duplicates)

contracts_overdue_tsal['CONTRACT_KEY'] = contracts_overdue_tsal['CONTRACT_KEY'].astype(str) 
# contracts_overdue_tsal.head(1)

257030


In [4]:
name = pre + "data/interm/test/overdue_contracts.pkl"
test_overdue_contracts = pd.read_pickle(name, compression={'method':'zip'})

print(len(test_overdue_contracts))
# test_overdue_contracts.head(1)

172054


In [5]:
%run ../init.ipynb #подгрузить нужные библиотеки

kwargs = dict( sourcesName = ['tsal', 'target'], 
               suffix = '_target',
               on = 'CONTRACT_KEY',  
               notnull_col = ['SNAPSHOT_DT', 'decision_date'],
               base_col = ['CONTRACT_NUM', 'CONTRACT_KEY', 'INN', 'decision_date']
             )

comp =  ComparisonDataFrames(contracts_overdue_tsal, test_overdue_contracts, **kwargs)   

In [6]:
missed = comp.missedRows(sourcesName='target')

отсутствуют в target 86392 строк


In [7]:
df = comp.presentRows()
# df.head(1)

одновременно присутствуют в tsal и target 170638 строк


полностью совпасть не сможем, так как платежи у нас после перерасчетов

In [8]:
df = comp.check_fields('OVERDUE_FLG', 'OVERDUE_FLG_target')
# df.head(1)

значения не совпадают у 308 строк из 170638


In [9]:
# df[df['OVERDUE_FLG']==1]['CONTRACT_KEY'].to_list()
# df[df['OVERDUE_FLG']==1]

In [10]:
df[df['OVERDUE_FLG']==0]

,CONTRACT_NUM,CONTRACT_KEY,INN,decision_date,OVERDUE_FLG,OVERDUE_FLG_target


Проверим, что это за платежи

In [11]:
def read_overdue_contracts():
    sql_query = """ SELECT DISTINCT PDD.SNAPSHOT_DT,
            PDD.CONTRACT_KEY,
            PDD.CONTRACT_APP_KEY,
            PDD.CBC_DESC,
            PDD.PAYMENT_NUM,
            PDD.PLAN_PAY_DT_ORIG,
            PDD.PLAN_AMT,
            PDD.PAYMENT_ITEM_KEY,
            PDD.CURRENCY_KEY,
            PDD.OFF_SCHEDULE, 
            FDD.MAX_PAY_DT_ORIG, 
            FDD.FACT_PAY_AMT
          FROM DM.DM_DETAILS_DAILY PDD
        LEFT JOIN
          (SELECT DISTINCT PBD.CONTRACT_KEY,
            PBD.CONTRACT_APP_KEY,
            CBC_DESC,
            PAYMENT_NUM,
            PLAN_PAY_DT_ORIG,
            PLAN_AMT ,
            MAX(PBD.PAY_DT_ORIG) OVER (PARTITION BY PBD.CONTRACT_KEY, PBD.CONTRACT_APP_KEY, CBC_DESC, PAYMENT_NUM, PLAN_PAY_DT_ORIG, PLAN_AMT, PAYMENT_ITEM_KEY, CURRENCY_KEY, OFF_SCHEDULE)         AS MAX_PAY_DT_ORIG ,
            SUM(NVL(PBD.FACT_PAY_AMT,0)) OVER (PARTITION BY PBD.CONTRACT_KEY, PBD.CONTRACT_APP_KEY, CBC_DESC, PAYMENT_NUM, PLAN_PAY_DT_ORIG, PLAN_AMT, PAYMENT_ITEM_KEY, CURRENCY_KEY, OFF_SCHEDULE) AS FACT_PAY_AMT ,
            PAYMENT_ITEM_KEY,
            CURRENCY_KEY,
            OFF_SCHEDULE
          FROM
            (SELECT CONTRACT_KEY,
              CONTRACT_APP_KEY,
              CBC_DESC,
              PAYMENT_NUM,
              PLAN_PAY_DT_ORIG,
              PLAN_AMT ,
              PAY_DT_ORIG,
              FACT_PAY_AMT,
              PAYMENT_ITEM_KEY,
              CURRENCY_KEY,
              OFF_SCHEDULE
            FROM DM.DM_DETAILS_DAILY
            WHERE PAYMENT_ITEM_KEY  IN (9, 1000)
            AND NVL(PLAN_AMT, 0)     > 0
            AND PRE_PAY              > 0
            ) PBD
          INNER JOIN
            ( SELECT * FROM PRE_CONTRACTS
            ) PRE_CON
          ON PBD.CONTRACT_KEY = PRE_CON.CONTRACT_KEY
          AND PBD.PAY_DT_ORIG < PRE_CON.DECISION_DATE - 1
          ) FDD
        ON PDD.CONTRACT_KEY                             = FDD.CONTRACT_KEY
        AND PDD.CONTRACT_APP_KEY                          = FDD.CONTRACT_APP_KEY
        AND PDD.CBC_DESC                                  = FDD.CBC_DESC
        AND PDD.PAYMENT_NUM                               = FDD.PAYMENT_NUM
        AND PDD.PLAN_PAY_DT_ORIG                          = FDD.PLAN_PAY_DT_ORIG
        AND PDD.PLAN_AMT                                  = FDD.PLAN_AMT
        AND PDD.PAYMENT_ITEM_KEY                          = FDD.PAYMENT_ITEM_KEY
        AND PDD.CURRENCY_KEY                              = FDD.CURRENCY_KEY
        AND PDD.OFF_SCHEDULE                              = FDD.OFF_SCHEDULE
      WHERE            PDD.PAYMENT_ITEM_KEY  IN (9, 1000)
            AND NVL(PDD.PLAN_AMT, 0)     > 0
            AND PDD.PRE_PAY              > 0 
      AND (PDD.PLAN_AMT - NVL(FDD.FACT_PAY_AMT, 0) >0
      OR  (PDD.PLAN_PAY_DT_ORIG < NVL(FDD.MAX_PAY_DT_ORIG,to_date ('01.01.2400', 'dd.mm.yyyy'))
      AND (EXTRACT (MONTH FROM PDD.PLAN_PAY_DT_ORIG) <> EXTRACT (MONTH FROM NVL(FDD.MAX_PAY_DT_ORIG,to_date ('01.01.2400', 'dd.mm.yyyy')))
      OR EXTRACT (YEAR FROM PDD.PLAN_PAY_DT_ORIG)      <> EXTRACT (YEAR FROM NVL(FDD.MAX_PAY_DT_ORIG,to_date ('01.01.2400', 'dd.mm.yyyy'))))))
      AND PDD.PLAN_PAY_DT_ORIG < to_date ('2023-07-05', 'yyyy-mm-dd')
      and PDD.CONTRACT_KEY IN (1078205,1553734,1627188,991126,638838,1063890,934599,950440,1568441,1039493,861008,1558182,886690,981040,1040348,1006855,1094029,1578928,990936,1647468,946263,964382,1633835,883095,1527049,1557727,987656,611598,1557725,987657,1086348,1099166,975566,1015444,1017319,1646160,1056949,1056754,1016691,946276,944197,1558230,1622078,1528271,1543070,964284,1591020,968914,1578887,1072483,970146,964476,987651,927612,1659587,1579045,1629158,1645487,1617198,990483,935598,866954,909034,1578984,1032417,1558416,1578937,946282,956163,985733,999032,1568077,1579005,909092,1639514,1644916,1673454,1579050,1633764,1611139,815652,822538,949455,1017324,981548,1633726,1632062,1016708,859667,1544703,1614418,1029708,1544852,886768,1617601,1619139,766030,1649182,1578916,1629159,944330,927617,1003961,953415,1647432,1614449,848420,1535609,941604,1017488,884331,990447,1017314,1658861,1659527,1559890,1558180,1579013,1617205,882455,1099812,1528296,951894,1634482,1611090,1632659,1647418,1042496,1518766,976603,1090378,848709,1523950,964370,1569599,1591667,990446,1042498,935567,1536922,1551936,1552820,1578451,1578944,1627182,1578929,1636715,1659608,1042969,881575,1631660,1673482,1079472,1558207,909110,940844,1610047,1617245,1556790,1574883,1578885,1094548,1647335,991190,1078122,1524495,1569665,1643843,1042508,1529005,976479,1047142,1566163,1578882,1029747,1554599,1633824,898858,701828,1012366,1029750,874213,1537176,982637,1658964,781526,946273,949454,986340,1093986,1642496,1547852,588033,1618706,1625255,1032414,1616822,976616,1570481,1094028,1578892,1578951,898859,874661,990941,1643086,1011204,1617158,1578943,991103,1626712,1577492,933212,912817,1058404,1615548,932450,1577899,1033617,793899,754372,1537283,1551973,1550286,1587495,1037093,1077109,1645620,1638595,946281,1633849,1625867,1093980,1557133,743151,934108,872381,1579919,1017917,1579009,861078,1532576,1614357,1532032,1641988,1620836,1015523,1004811,1579057,935676,1611074,1647375,945892,1003911,1015538,1624087,990574,1608597,1017416,1568537,1602391,946264,877321,1013218,1629163,991189,1092044,948620,1520437,881582,1624518,909002,906609,1558252,1640539,1093902,935047,1017919,824494,1087339,877292,768660,1562062,1640521,1017951,1632661,1004822,1635677,1640524,881537,741101,1659626,1568027,1658816,989360,1014387,909072,1017930,1608599,659007,1611129,946959,1640532,1013202)

                """
                
    df = read_from_oracle(sql_query, title='diff')
    
    return df

diff = read_overdue_contracts()
# diff.head(1)

Размер считанных данных diff - (4986, 12)


In [12]:
# diff['PLAN_PAY_DT_ORIG'].unique()

все они должны были быть оплачены 2023-07-04, но на 05.07.23 еще в цал данные об оплате не пришли, поэтому у них стоит просрочка, хотя на ее нет